<a href="https://colab.research.google.com/github/scskalicky/LING-226-vuw/blob/main/34_Word_Vector_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Vectors

You might not have any idea what someone means by the term word vectors or word embeddings, but such technology can no longer be escaped. What we will cover in this notebook is a method to represent word relationships in a new way, one which has eventually formed the backbone of generative AI technology such as ChatGPT.

Now, we are not going to be training ChatGPT models, but we are going to look at how to model the distributional patterns of words using algorithms which generate word vectors. One notable algorithm is `word2vec`. What is so great about it? Well, rather than calculating raw probabilities of word occurences, word2vec considers how words influence the probabilities of other words, and then groups similar words together based on those probabilities. It does so by assigning words not just one score, but tens or hundreds of scores which represent how the words function in a bunch of different contexts. These scores are the vectors, which can be mapped to a multidimension space. Then, the distance among these scores can be compared in the space to find similar / different words.

There are ton of attempts to explain word2vec. One article called [The Illustrated word2vec](https://jalammar.github.io/illustrated-word2vec/). might be of interest to you, as well as the [author's YouTube video explaining the article](https://www.youtube.com/watch?v=ISPId9Lhc1g).

Maybe you don't care too much about this, or maybe my explanation sucks. The point is, that we can learn how to use this algorithm to train our very own vector models on our texts. Then we can find which words are most similar to other words in texts. To do so, we will use a library called [`gensim`](https://radimrehurek.com/gensim/).

Let's use the `state_union` corpus in `nltk` to learn how to train and use a model.



In [ ]:
# need to tokenize plus download state_union corpus
import nltk
nltk.download(['punkt', 'state_union'])
from nltk.corpus import state_union

We need to convert our data into a list of tokenized "sentences", although the term "sentence" is a big vague here. A sentence could be a paragraph or an entire document. Below I will convert each State of the Union text into a single tokenized list, resulting in a list of lists (which is what the gensim Word2Vec will expect as input).

I will also lower case the words and remove any numbers and punctuation.

In [ ]:
# output containers, plus corpus size for info
state_union_combined = []
corpus_size = 0

# loop the the file ids then tokenize and add to list
for fileid in state_union.fileids():
  text = state_union.raw(fileid)
  words = nltk.word_tokenize(text)
  words = [word.lower() for word in words if word.isalpha()]
  state_union_combined.append(words)
  corpus_size += len(words)

# how many words in the corpus?
print(corpus_size)

## **state union word2vec model**

To fit a `word2vec` model, we choose the text, plus the `vector_size`, `epohcs`, `window`, and `min_count`.

The default vector size is 100, which means each word in the text will have 100 values or vectors (these are the values used to represent the word in the multidimensional space). I will lower this to 50 because of our relatively small corpus.  `window` represents the size of the text spans the algorithm will use (i.e., it looks at each sequential 5-word contents). `min_count` means a word must occur at least that many times, 5 is the default but I will increase that to ten.

`epochs` represents the number of retrainings to do, which can help when you have small data. I ask for 10 epochs here as a means to obtain a more stable model.

These parameters are not set in stone, and usually require some tweaking or adjustment based on the size of the corpus. So, feel free to play around with them


Our state union corpus is pretty small (only about 344k words), but we will still be able to find some interesting association among words.

In [ ]:
# load in gensim and the word2vec class
import gensim
from gensim.models import Word2Vec

In [ ]:
# train the model using our texts and specific parameters
su_w2v = Word2Vec(sentences = state_union_combined, vector_size = 50, epochs = 10, window = 5, min_count = 10)

The model trains relatively fast in even in Colab. Now that we have a trained model, what can we do with it? With these vectors, we can:


1. query the model to find the most similar words to an input word
2. find the difference between different words
3. find words that don't match in a list of other words


### **`.most_similar()`**

The most useful information we can obtain from a model like this is word similarity metrics. We can ask the model to output a list of the most similar words to an input word. By default, this will produce the top twn most similar words, alone with the cosine distance value. The cosine distance value will range from -1 to 1. Negative values mean the words mean something very different, positive values mean the words mean something similar, and neutral values indicate no relationship.

> By the way, this is exactly what `spaCy` does when using the `.similar()` function to find the similarity values between two words.

To use `.most_similar()`, we need to access the `.wv` attribute from the model, which stands for word vectors. Below I look up the similarity metrics for several words. What do you think about the output?



In [ ]:
su_w2v.wv.most_similar('america')

In [ ]:
su_w2v.wv.most_similar('conflict')

In [ ]:
su_w2v.wv.most_similar('peace')

In [ ]:
su_w2v.wv.most_similar('solution')

### **`.most_similar(positive, negative)`**

You can be more specific in your approach to using `.most_similar()`, by supplying not just one word, but several words. You can also specify whether the similarity should be positively or negatively related to some words.

Compare the difference between asking for only the most similar words to `"money"` against asking for words similar to `"money"` but not similar to `"war"`.



In [ ]:
su_w2v.wv.most_similar(positive = 'money')

In [ ]:
su_w2v.wv.most_similar(positive = 'money', negative = 'war')

You can enter as many words as you like as a list.

In [ ]:
su_w2v.wv.most_similar(positive = ['citizen', 'taxpayer'])

In [ ]:
#slight differences here.
su_w2v.wv.most_similar(positive = ['citizen', 'taxpayer'], negative = ['terrorist'])

In [ ]:
su_w2v.wv.most_similar(positive = ['jobs'])

In [ ]:
su_w2v.wv.most_similar(positive = ['jobs'], negative = ['military'])

### **.doesnt_match()**

This function takes a list of words and tells you which word is the least similar to the other words. I am not sure how useful this really is, but it is at least fun to play with.

In [ ]:
su_w2v.wv.doesnt_match(['war', 'jobs', 'military'])

In [ ]:
# why do you think "america" is the outlier here?
su_w2v.wv.doesnt_match(['america', 'france', 'germany', 'china'])

In [ ]:
su_w2v.wv.doesnt_match(['rights', 'constitution', 'freedom', 'taxes'])

## **fastext**

Word2Vec is not the only way to train word vectors / embeddings. Another method is called FastText. [This algorithm works a bit differently by breaking down words and considering morphological differences.](https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html#sphx-glr-auto-examples-tutorials-run-fasttext-py) It is also apparently better at dealing with mispelled words.

Let's try it out. First we import the required class. Then we create the model with a vector size of 50.

Then, we build the model vocabulary using the same `state_union_combined` object we fed to word2vec above.



In [ ]:
from gensim.models.fasttext import FastText
su_ft = FastText(vector_size = 50)

# build the vocabulary
su_ft.build_vocab(corpus_iterable = state_union_combined)

Now we need to train the model. There are a number of parameters, which we will leave as defaults.

In [ ]:
#train the model
su_ft.train(corpus_iterable = state_union_combined, epochs = su_ft.epochs,
                  total_examples = su_ft.corpus_count, total_words = su_ft.corpus_total_words)

Now we can use the model just the same as the word2vec model to explore similarities among words. Compare the output here versus the output for word2vec. The most similar words here are more similar in terms of morphological form. This is because of differences in the algorithms.

In [ ]:
# fastText vectors
su_ft.wv.most_similar("america")

In [ ]:
# word2vec vectors
su_w2v.wv.most_similar("america")

## **using pretrained vectors**

You can also use sets of pretrained vectors which have been run on much larger data sets. The language models in `spaCy` for example have larger vectors than the data we have here. Keep in mind those vectors are intending to show similarity at a generalizable level. One advantage of training on our own data is that we can explore similarity in a specific context.

One interesting experiment then is to see whether your corpus or texts have different similarity relationships when compared to a larger set of reference vectors.

`gensim` has [a number of pretrained models you can use.](https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models).



In [ ]:
# all gensim models:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

In the cell below, I ask for gensim to download a pretrained set of vectors trained on a corpus of 3 million words from google news. The dimensions of this set are 300, meaning there are 300 vectors for each word. The model is 1.6 GB large, so it takes awhile to download. If you want to reuse resources such as this, it would be best to download it to your Google Drive and then reload it as you need. But if you want to click Play below, go grab a drink while you want for it to download.

You would need to repeat this download each time you run the notebook.

In [ ]:
# download

google_w2v = glove_vectors = gensim.downloader.load('word2vec-google-news-300')

Now that we have a loaded model, we can use it just the same. Because we have downloaded the vectors directly and not the model, we do not need to use the `.wv.` part.

You can see the differences using `type`

In [ ]:
# a model
type(su_w2v)

In [ ]:
# keyed vectors
type(su_w2v.wv)

In [ ]:
# keyed vectors
type(google_w2v)

Let's compare the most similar words to 'america' in the Google New corpus vectors. I will show the state union vectors again for reference. We can see that we obtain vastly different results. This is a direct result of the different training data use. The Google News dataset is larger and has more topics. The State Union corpus is smaller and more homogenous (i.e., it is all speeches from US Presidents).


In [ ]:
google_w2v.most_similar('america')

In [ ]:
su_w2v.wv.most_similar('america')

# **Conclusion**

Training word vectors is a bit more complicated than the other NLP tasks covered so far. But, libraries like `gensim` make it relatively painless to do so. It is very important to explore the output of the model and potentially tune the different parameters until you get something that makes sense.

Take this time to either train your own model or explore the ones I've loaded / created here, or to look into the other models provided by `gensim`.

There are also other sources of pretrained vectors besides `gensim`, and you can use `gensim` to load in these vectors and use them as you like.

You might still be a bit fuzzy about what vectors actually *are*, and that's pretty normal. [There are several blogs and other explanations](https://cloud.google.com/blog/topics/developers-practitioners/meet-ais-multitool-vector-embeddings) out there which may help. Vectors are now fundamental tools for modern NLP and AI, so gaining an awareness of how they work and how to train them is important.